In [47]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import seaborn as sns
import glob
import cv2
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import models
from keras.models import Sequential

In [48]:
def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        if root.find('object'):
            for member in root.findall('object'):
                bbx = member.find('bndbox')                
                xmin = round(float(bbx.find('xmin').text))
                ymin = round(float(bbx.find('ymin').text))
                xmax = round(float(bbx.find('xmax').text))
                ymax = round(float(bbx.find('ymax').text))
                label = member.find('name').text
                value = (str(root.find('filename').text),
                        int(root.find('size')[0].text),
                        int(root.find('size')[1].text),
                        str(label),
                        int(xmin),
                        int(ymin),
                        int(xmax),
                        int(ymax)
                        )
                xml_list.append(value)

    column_name = ['filename', 'width', 'height',
                   'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xmlDf = pd.DataFrame(xml_list, columns=column_name, index=None)
    return xmlDf

In [49]:
df = xml_to_csv("data/annotations/")
print(df.head())

train = df.iloc[:(int)(len(df) * 0.7), :]
train.to_csv("data/train_data.csv", index=None)
test = df.iloc[(int)(len(df) * 0.3):, :]
test.to_csv("data/test_data.csv", index=None)

training_image_label = pd.read_csv("data/train_data.csv")
train_images = []
train_targets = []
train_labels = []

testing_image_label = pd.read_csv("data/test_data.csv")
test_images = []
test_targets = []
test_labels = []

      filename  width  height    class  xmin  ymin  xmax  ymax
0  Cars213.png    400     300  licence   150   160   233   194
1  Cars207.png    400     478  licence   254   229   372   264
2  Cars398.png    400     300  licence    72   139   196   222
3  Cars367.png    450     299  licence   224   161   382   222
4  Cars401.png    400     300  licence   158   223   238   240


In [50]:
for index, row in training_image_label.iterrows():
    (filename, width, height, class_name, xmin, ymin, xmax, ymax) = row
    
    train_image_fullpath = os.path.join("data/images/", filename)
    image = Image.open(train_image_fullpath)
    train_img = image.resize((128, 128))
    train_img = train_img.convert('RGB')
    train_img_arr = np.array(train_img) 

    # Bounding Box Coordinate
    xmin = round(xmin/ width, 2)
    ymin = round(ymin/ height, 2)
    xmax = round(xmax/ width, 2)
    ymax = round(ymax/ height, 2)
    
    train_images.append(train_img_arr)
    train_targets.append((xmin, ymin, xmax, ymax))
    train_labels.append(class_name)
    
for index, row in testing_image_label.iterrows():
    (filename, width, height, class_name, xmin, ymin, xmax, ymax) = row
    
    test_image_fullpath = os.path.join("data/images/", filename)
    image = Image.open(test_image_fullpath)
    test_img = image.resize((128, 128))
    test_img = test_img.convert('RGB')
    test_img_arr = np.array(test_img) 
    print(test_img_arr.shape)
    
    # Bounding Box Coordinate
    xmin = round(xmin/ width, 2)
    ymin = round(ymin/ height, 2)
    xmax = round(xmax/ width, 2)
    ymax = round(ymax/ height, 2)
    
    test_images.append(test_img_arr)
    test_targets.append((xmin, ymin, xmax, ymax))
    test_labels.append(class_name)
    

(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 

In [51]:
print(type(train_images))

train_images = np.asarray(train_images, dtype="uint8")
train_targets = np.array(train_targets)
train_labels = np.array(train_labels)
validation_images = np.asarray(test_images, dtype="uint8")
validation_targets = np.array(test_targets)
validation_labels = np.array(test_labels)

print(train_images.shape)




<class 'list'>
(329, 128, 128, 3)


In [52]:
num_classes = 2
model = Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=input_shape),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(64, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation="sigmoid")
])


In [57]:
model.compile(loss="mean_squared_error", optimizer='Adam', metrics=['accuracy'])


history = model.fit(train_images, train_targets, epochs=5)

Epoch 1/5


ValueError: in user code:

    File "/Users/jasonwang/anaconda3/lib/python3.11/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/Users/jasonwang/anaconda3/lib/python3.11/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/jasonwang/anaconda3/lib/python3.11/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/Users/jasonwang/anaconda3/lib/python3.11/site-packages/keras/engine/training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/jasonwang/anaconda3/lib/python3.11/site-packages/keras/engine/training.py", line 1109, in compute_loss
        return self.compiled_loss(
    File "/Users/jasonwang/anaconda3/lib/python3.11/site-packages/keras/engine/compile_utils.py", line 265, in